In [ ]:
# this is classification of mnist data through lstm model
# this is just for the sake of understanding lstm better

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# data is into train, test, and validation
# each image is hving a label for digit 0-9, but here we r taking label in one hot encoding

mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)

In [ ]:
# setting hyper parameters for our classifier model
lr = 0.001
batchSize = 128
epochs = 100000

In [ ]:
# here in the model i am using 2 layers and one lstm cell structure.
# the 1st hidden layer is input layer, then lstm cell, and then hidden output layer 

#here as we r using self made hidden layer for input before lstm cell, so we have to
# give them weights, biases and manipulate input dimensions n features as we want to feed lstm with

# We know that Lstm (rnn) processes sequential data, so to fit the mnist image data in lstm
# the mnist image is a matrix of 28*28, hence to feed in lstm, we consider this image as a 
# 28 rows where each row is having 28 units or points in it.

In [ ]:
# giving parameters for layers
columns = 28
# this represents the no. of colun in mnist image
timeSteps = 28
# the rows that we aconsidering can be thought of as time steps in lstm's

hiddenUnits = 128
# this is the no. of neurons that will be in the hidden layer

classes = 10
# as we classify it into 0-9 (10 classses)

In [ ]:
# making placeholders for lstm network
# here the placeholder can be considered as 3 dimensional
# 1st dimensionn as usual none to accept the batch size,
# 2nd dimension gives how many rows or time steps
# the 2nd dimension represents timestep, so the no of time steps = no. of lstm cells
#3rd dimension gives how many points (analogous to words) in each row

# in output y placeholder, we are just having the output class from 0-9 in
# one hot encoding
# 1st dimension usual none, for batch size
# 2nd dimension 10 for one hot encoding

x = tf.placeholder(tf.float32, [None, timeSteps, columns])

y = tf.placeholder(tf.float32, [None, classes])


In [ ]:
# now for making weights and biases of input layer
# we know that input layer is hving 128 neurons, and the no of input features to these neurons 
# will be the no. of columns image is having that is 28

inputLayerWeights = tf.Variable(tf.random_normal([columns, hiddenUnits]))
# as biases need not be updated we can keep that in constant
inputLayerBiases = tf.constant(0.1, shape = [hiddenUnits, ])

In [ ]:
# output layer weights n biases
outputLayerWeights = tf.Variable(tf.random_normal([hiddenUnits, classes]))
#as output layer has 128 neurons and output into 10 classes
outputLayerBiases = tf.constant(0.1, shape = ([classes, ]))

In [ ]:
# now we will define fn to compile all layers together including BasicLstm
def myLstmNetwork(inputs, weights, bias):
    
    #to prepare input layer we have to reshape the data coming from input placeholder
    #coz weights are in diff format 
    # X ==> (128 batch * 28 steps, 28 inputs)
    # -1 is used in reshape to either flatten or infer shape, look doc for more
    X = tf.reshape(inputs, [-1, columns])
    # the column will remain the same shape as it is feature
    Xin = tf.matmul(X, weights) + bias
    # Xin will b the output of input layer from the 128 hidden neurons
    # X_in ==> (128 batch * 28 steps, 128 hidden)
    
    # now we again reshape it into batches and make it 3d
    
    Xout = tf.reshape(Xin, [-1, timeSteps, columns])
    
    # now we will make lstm cell
    
    cell = tf.contrib.rnn.BasicLSTMCell(hiddenUnits, forget_bias = 1.0, state_is_tuple = True)
    
    # lstm cell is divided into two parts (c_state, h_state)
    # look for cell state in tensorflow
    initState = cell.zero_state(batchSize, dtype=tf.float32)
    # we have initialized the cell with first zero state with batch size
    
    #now we will built our complete rnn with these cells 
    # we will use dynamic rnn than rnn, in rnn it creates static length rnn, say if we create net having 100 
    #timesteps it will  fixed.(slower also)
    # using dynamic rnn it dynamically calculates the time steps with its while loop
    
    ouputs,finalState = tf.nn.dynamic_rnn(cell, Xout, initial_state = initState, time_major = False)
    
    # here our time_major false means the first dimension of the input is not time step. here it is 
    # batch size
    #final state is the ouput from the final cell i guess
    
    #finalState will b returned in tuple format, (cstate, hstate)
    #we r interested only in final state coz this is a memory lstm
    
    results = tf.matmul(finalState[1], outputLayerWeights) + outputLayerBiases
    
    return results

In [ ]:
prediction = myLstmNetwork(x, inputLayerWeights, inputLayerBiases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))

optimizer = tf.train.AdamOptimizer(lr)
train = optimizer.minimize(cost)

In [ ]:
# calculating accuracy

corrPrediction = tf.equal(tf.argmax(prediction, axis = 1), tf.argmax(y, axis = 1))
accuracy = tf.reduce_mean(tf.cast(corrPrediction, tf.float32))


In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
